In [1]:
import numpy as np 
import string 
from nltk.corpus import stopwords

* **"""Compute softmax values for each sets of scores in x."""**

In [2]:
def softmax(value):
    exponential_value = np.exp(value - np.max(value))
    return exponential_value / exponential_value.max()

* Wordvector Class

In [11]:
class word_to_vector(object):
    def __init__(self) :
        self.values = 10
        self.x_train = []
        self.y_train = []
        self.window_size = 2
        self.learning_rate = 0.001
        self.words = []
        self.word_index = {}
        #-----------------------------------------------
    def initialize(self,v,data):
        self.v = v
        self.w = np.random.uniform(-0.8,0.8,(self.v,self.values))
        self.w1= np.random.uniform(-0.8,0.8,(self.values , self.v))
        self.words = data 
        for i in range(len(data)):self.word_index[data[i]] = i 
        #----------------------------------------------------------
        #Create Data Forward Propagation Function 
    def feed_forward(self,x):
        self.h = np.dot(self.w.T,x).reshape(self.values,1)
        self.u = np.dot(self.w1.T,self.h)
        print(self.u)
        self.y = softmax(self.u)
        return self.y 
        #----------------------------------------------------------------
        #Create BackPropagatiion Function Because Make Update Weights
    def backpropagate(self,x,t):
        e = self.y - np.asarray(t).reshape(self.values,1)
        dervative_w1 = np.dot(self.h,e.T)
        X = np.array(x).reshape(self.values,1)
        dervative_w = np.dot(X , np.dot(self.w1,e).T)
        self.dervative_w1 = self.learning_rate * dervative_w1
        self.dervative_w  = self.learning_rate * dervative_w
        #----------------------------------------------------------------------
    def train(self,epochs): 
        for x in range(1,epochs+1):		 
            self.loss = 0
            for j in range(len(self.x_train)): 
                self.feed_forward(self.x_train[j]) 
                self.backpropagate(self.x_train[j],self.y_train[j]) 
                C = 0
                for m in range(self.values): 
                    if(self.y_train[j][m]): 
                        self.loss += -1*self.u[m][0] 
                        C += 1
                self.loss += C * np.log(np.sum(np.exp(self.u)))
            print('epoch' ,x , 'Loss = ',self.loss)
            self.learning_rate *= 1/(( 1+ self.learning_rate * x))
    def predict(self,word,number_of_predictions): 
        if word in self.words: 
            index = self.word_index[word] 
            x= [0 for i in range(self.values)] 
            x[index] = 1
            prediction = self.feed_forward(x) 
            output = {} 
            for i in range(self.values): 
                output[prediction[i][0]] = i 
            top_context_words = [] 
            for k in sorted(output,reverse=True):
                top_context_words.append(self.words[output[k]])   
                if (len(top_context_words)>=number_of_predictions): break 
            return top_context_words 
        else : print ("Word not found in dicitonary")

In [4]:
'''class word2vec(object): 
	def __init__(self): 
		self.N = 10
		self.x_train = [] 
		self.y_train = [] 
		self.window_size = 2
		self.alpha = 0.001
		self.words = [] 
		self.word_index = {} 

	def initialize(self,V,data): 
		self.V = V 
		self.W = np.random.uniform(-0.8, 0.8, (self.V, self.N)) 
		self.W1 = np.random.uniform(-0.8, 0.8, (self.N, self.V)) 
		
		self.words = data 
		for i in range(len(data)): 
			self.word_index[data[i]] = i 

	
	def feed_forward(self,X): 
		self.h = np.dot(self.W.T,X).reshape(self.N,1) 
		self.u = np.dot(self.W1.T,self.h) 
		#print(self.u) 
		self.y = softmax(self.u) 
		return self.y 
		
	def backpropagate(self,x,t): 
		e = self.y - np.asarray(t).reshape(self.V,1) 
		# e.shape is V x 1 
		dLdW1 = np.dot(self.h,e.T) 
		X = np.array(x).reshape(self.V,1) 
		dLdW = np.dot(X, np.dot(self.W1,e).T) 
		self.W1 -=  self.alpha*dLdW1 
		self.W -= self.alpha*dLdW 
		
	def train(self,epochs): 
		for x in range(1,epochs+1):		 
			self.loss = 0
			for j in range(len(self.X_train)): 
				self.feed_forward(self.X_train[j]) 
				self.backpropagate(self.X_train[j],self.y_train[j]) 
				C = 0
				for m in range(self.V): 
					if(self.y_train[j][m]): 
						self.loss += -1*self.u[m][0] 
						C += 1
				self.loss += C*np.log(np.sum(np.exp(self.u))) 
			print("epoch ",x, " loss = ",self.loss) 
			self.alpha *= 1/( (1+self.alpha*x) ) 
			
	def predict(self,word,number_of_predictions): 
		if word in self.words: 
			index = self.word_index[word] 
			X = [0 for i in range(self.V)] 
			X[index] = 1
			prediction = self.feed_forward(X) 
			output = {} 
			for i in range(self.V): 
				output[prediction[i][0]] = i 
			
			top_context_words = [] 
			for k in sorted(output,reverse=True): 
				top_context_words.append(self.words[output[k]]) 
				if(len(top_context_words)>=number_of_predictions): 
					break
	
			return top_context_words 
		else: 
			print("Word not found in dicitonary") '''

3. **Make Data Preprocessing**

In [12]:
def preprocessing(corpus_data):
    stop_words = set(stopwords.words('english'))
    training_data = []
    sentences = corpus_data.split(".")
    for i in range (len(sentences)):
        sentences[i] = sentences[i].strip()
        sentence = sentences[i].split()
        x_value = [word.strip(string.punctuation) for word in sentence if word not in stop_words]
        x_value = [word.lower() for word in x_value]
        training_data.append(x_value)
    return training_data

4. Prepare Data For Training

In [13]:
def prepare_data(sentences,w2v):
    data ={}
    for sentence in sentences:
        for word in sentence:
            if word not in data: data[word] =1
            else : data[word] +=1
    value = len(data)
    data = sorted(list(data.keys()))
    vocab = {}
    for i in range(len(data)) : 
        vocab[data[i]] = i 
    for sentence in sentences :
        for i in range(len(sentences)):
            center_word = [0 for x in range (value)]
            center_word[vocab[sentence[i]]] = 1
            context = [ 0 for x in range (value)]
            for j in range(i-w2v.window_size , i + w2v.window_size):
                if i != j and j >= 0 and j<len(sentence):context[vocab[sentence[j]]] +=1
            w2v.x_train.append(center_word)
            w2v.y_train.append(context)
    w2v.initialize(value,data)
    return w2v.x_train,w2v.y_train

5. Detect Training Data & Number of epochs
6. Send Corpous Data --> because make preprocessing

In [14]:
corpus = "The earth revolves around the sun. The moon revolves around the earth"
epochs = 1000 
training_data = preprocessing(corpus)
training_data

[['the', 'earth', 'revolves', 'around', 'sun'],
 ['the', 'moon', 'revolves', 'around', 'earth']]

7. Recall Word_to_vector class
8. prepare data

In [15]:
w2v =word_to_vector()
prepare_data(training_data , w2v)

([[0, 0, 0, 0, 0, 1],
  [0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1],
  [0, 0, 1, 0, 0, 0]],
 [[0, 1, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 1],
  [0, 0, 1, 0, 0, 0],
  [0, 0, 0, 1, 0, 1]])

9. Train Model

In [16]:
w2v.train(epochs)

[[-0.22779061]
 [ 0.26483157]
 [ 0.28919225]
 [-0.64472373]
 [-0.99836706]
 [ 0.0757863 ]]


ValueError: cannot reshape array of size 6 into shape (10,1)

In [17]:
print('w2v.predict("around",3) --> ',w2v.predict("around",3))
print('w2v.predict("around",5) --> ',w2v.predict("around",5))
print('w2v.predict("sun",3) --> ', w2v.predict("sun",3))
print('w2v.predict("earth",3) --> ', w2v.predict("earth",3))
print('w2v.predict("jupiter",3)--> ', w2v.predict("jupiter",3))

ValueError: shapes (10,6) and (10,) not aligned: 6 (dim 1) != 10 (dim 0)